### Temporary views are logical SQL views created from DataFrames or tables.
They allow you to query data using SQL syntax without saving permanent metadata in the metastore.
TYPES OF TEMPORARY VIEWS:
1. LOCAL TEMP VIEW:
   - Created using: df.createOrReplaceTempView("view_name")
   - Scope: Visible only within the current Spark session (notebook or cluster session).
   - Automatically removed once the session ends.
2. GLOBAL TEMP VIEW:
   - Created using: df.createOrReplaceGlobalTempView("view_name")
   - Scope: Visible across all sessions and notebooks running on the same Databricks cluster.
   - Stored under a special database: global_temp.
   - Must be referenced as global_temp.view_name.
   - The view persists until the cluster is terminated.
USE CASES:
- Share temporary transformed datasets between notebooks in the same cluster.
- Debug or inspect intermediate results without creating permanent tables.

**Note:**
1. You can access global temp views from any other notebook in the same cluster:<br>
       _spark.sql('SELECT * FROM global_temp.sales_summary').show()_
2. Local temp views cannot be accessed outside the current session.
3. Both temp and global temp views are dropped automatically when the session or cluster ends.

In [0]:
# Step 1: Create a sample DataFrame
data = [
    (1, "East", "Laptop", 1200.50),
    (2, "West", "Mobile", 800.75),
    (3, "South", "Tablet", 450.25),
    (4, "East", "Monitor", 300.00)
]
columns = ["sale_id", "region", "product", "amount"]

df = spark.createDataFrame(data, columns)

# Step 2: Write DataFrame as a Delta table
df.write.format("delta").mode("overwrite").saveAsTable("inceptez_catalog.inputdb.sales_data")

# Step 3: Read the table back into a DataFrame
sales_df = spark.table("inceptez_catalog.tempdb.sales_data")

# Step 4: Create a GLOBAL TEMP VIEW
sales_df.groupBy("region").agg(F.sum("amount").alias("total_sales")).createOrReplaceGlobalTempView("sales_summary")

# Step 5: Query the GLOBAL TEMP VIEW
print("GLOBAL TEMP VIEW DATA:")
display(spark.sql("SELECT * FROM global_temp.sales_summary"))

# Step 6: Create a LOCAL TEMP VIEW for comparison
sales_df.groupBy("region").agg(F.count("*").alias("total_orders")).createOrReplaceTempView("sales_local")

print("LOCAL TEMP VIEW DATA:")
display(spark.sql("SELECT * FROM sales_local"))

# Step 7: Show all available global temp views
print("GLOBAL TEMP VIEWS IN 'global_temp' DATABASE:")
display(spark.sql("SHOW TABLES IN global_temp"))

# Step 8: Show all local temp views in current session
print("LOCAL TEMP VIEWS IN CURRENT SESSION:")
display(spark.sql("SHOW TABLES"))